### Juan Diego Martínez (201822347), Nicolás De La Hoz (201729392), María Sofía Álvarez López (201729031)

In [1]:
!pip install numpy==1.18.5
!pip install pandas==1.1.5

In [2]:
# Librerias generales
# Pandas
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import pandas_profiling
# Ranom seed
import numpy as np
np.random.seed(3301)

# Seaborn
import seaborn as sns 

# Matplolib
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# Ubicacion de la base de datos
db_location = "data/datos_SaludAlpes_diagnosticos_dataset.csv"
# Leer los datos
df_salud = pd.read_csv(db_location, delimiter=';')
# Dimensiones de los datos
print(df_salud.shape)

(768, 11)


In [4]:
profiling =pandas_profiling.ProfileReport(df_salud)
profiling.to_file("salud_profile.html")

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Ver los datos
display(df_salud.sample(5)) # Muestra de aleatoria de elementos

,Hair color,Pregnancies,Glucose,City,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
235,Black,4,171,New York,72,0,0,436,479,26,1
295,Black,6,151,New York,62,31,120,355,692,28,0
144,Black,4,154,New York,62,31,284,328,237,23,0
292,Black,2,128,New York,78,37,182,433,1224,31,1
60,Black,2,84,New York,0,0,0,0,304,21,0


In [6]:
# Tipos de datos de las columnas
df_salud.dtypes

Hair color                  object
Pregnancies                 object
Glucose                     object
City                        object
BloodPressure               object
SkinThickness               object
Insulin                     object
BMI                          int64
DiabetesPedigreeFunction    object
Age                          int64
Outcome                     object
dtype: object

Queremos conocer los distintos valores que pueden tomar las variables categóricas.

In [7]:
df_salud.City.value_counts().to_frame()

,City
New York,767
-,1


In [8]:
df_salud['Hair color'].value_counts().to_frame()

,Hair color
Black,686
Red,50
Blue,32


Vemos que hay algunos datos cuyas ausencias están descritas por guiones (-). Queremos saber cuantas filas tienen guiones, para ver si es un problema de todo el conjunto de datos.

In [9]:
a = df_salud.isin(['-']).sum(axis=1)

In [10]:
df_salud[df_salud.isin(['-']).any(axis=1)]

,Hair color,Pregnancies,Glucose,City,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
583,Black,-,-,-,-,-,-,387,-,42,-


Es solamente una fila la que tiene guiones. Podemos quitarla sin problema del conjunto de datos.

In [11]:
df_salud = df_salud.drop(np.where(a > 0)[0][0])

Pensamos inicialmente en quitar las columnas Ciudad y Hair Color pues no son relevantes para nuestro modelo. La ciudad, porque todos son de Nueva York (excepto un dato, que tiene -) y el color de pelo, porque no es el feature más relevante para diagnosticar la diabetes.

In [12]:
df_salud = df_salud.drop(['Hair color', 'City'], axis=1)

Veamos una muestra de como quedó nuestro data_frame,

In [13]:
df_salud.sample(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
223,7,142,60,33,190,288,687,61,0
763,10,101,76,48,180,329,171,63,0
451,2,134,70,0,0,289,542,23,1
686,3,130,64,0,0,231,314,22,0
661,1,199,76,43,0,429,1394,22,1


Revisemos los tipos de datos de nuestras columnas,

In [14]:
df_salud.dtypes

Pregnancies                 object
Glucose                     object
BloodPressure               object
SkinThickness               object
Insulin                     object
BMI                          int64
DiabetesPedigreeFunction    object
Age                          int64
Outcome                     object
dtype: object

Como vemos, a pesar de ser todas variables numéricas, Pandas las toma como object (por el dato con guiones). Debemos convertir estos datos a <code>float</code>, siguiendo el diccionario.

In [15]:
for columna in df_salud.columns:
    if columna not in ['Age', 'Outcome'] :
        df_salud[columna] = df_salud[columna].astype(float)

In [16]:
df_salud.dtypes

Pregnancies                 float64
Glucose                     float64
BloodPressure               float64
SkinThickness               float64
Insulin                     float64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                      object
dtype: object

In [17]:
df_salud

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,0.0,336.0,627.0,50,1
1,1.0,85.0,66.0,29.0,0.0,266.0,351.0,31,0
2,8.0,183.0,64.0,0.0,0.0,233.0,672.0,32,1
3,1.0,89.0,66.0,23.0,94.0,281.0,167.0,21,0
4,0.0,137.0,40.0,35.0,168.0,431.0,2288.0,33,1
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,329.0,171.0,63,0
764,2.0,122.0,70.0,27.0,0.0,368.0,34.0,27,0
765,5.0,121.0,72.0,23.0,112.0,262.0,245.0,30,0
766,1.0,126.0,60.0,0.0,0.0,301.0,349.0,47,1


In [18]:
number_cols = df_salud.select_dtypes(include = ['int64','float64']).columns

In [19]:
df_salud[number_cols].describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.839635,120.921773,69.096480,20.563233,79.903520,289.670143,432.395046,38.006519
std,3.368429,31.984561,19.366833,15.945349,115.283105,116.780873,336.144934,117.902397
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,251.500000,205.500000,24.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,309.000000,337.000000,29.000000
75%,6.000000,140.500000,80.000000,32.000000,127.500000,359.000000,592.000000,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,671.000000,2329.000000,3256.000000


Podemos ver que hay varios datos en cero que no tienen sentido. Es imposible tener la glucosa y la insulina en 0, por ejemplo. Pensamos que, para los datos ausentes, rellenaron con 0. 

In [20]:
df_salud.isin([float(0)]).sum(axis=0)

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               226
Insulin                     373
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

La mejor decisión es quitar estos datos. (Aquí, justificamos que es porque poner la media o algo así puede ser malo para los datos. Es decir, como tenemos datos desbalanceados, el modelo puede quedar mal).

<b>PREGUNTAR:</b> ¿Dónde buscamos de la función de Pedigree? No encontramos nada.
<b>PREGUNTAR:</b> ¿Sí es buena idea borrarlos?

In [21]:
indices = df_salud[(df_salud['Insulin'] == 0) | (df_salud['BMI'] == 0) | (df_salud['Glucose'] == 0) | (df_salud['BloodPressure'] == 0)  | (df_salud['SkinThickness'] == 0)].index
df_salud.drop(indices, inplace=True)

In [22]:
df_salud[number_cols].describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,308.017857,480.663265,39.117347
std,3.211424,30.860781,12.496092,10.516424,118.841690,105.863577,340.370137,163.210975
min,0.000000,56.000000,24.000000,7.000000,14.000000,24.000000,4.000000,21.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,262.750000,245.750000,23.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,328.000000,422.000000,27.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,368.250000,667.750000,36.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,671.000000,2329.000000,3256.000000


In [23]:
df_salud.isin([float(0)]).sum(axis=0)

Pregnancies                 56
Glucose                      0
BloodPressure                0
SkinThickness                0
Insulin                      0
BMI                          0
DiabetesPedigreeFunction     0
Age                          0
Outcome                      0
dtype: int64

<b>NOTAS: </b> 
* El BMI está multiplicado por 10.
* Preguntar lo de la presión arterial en el diccionario.


Ahora, vemos que hay un dato de edad en 3256. Podemos quitarlo, porque claramente no tiene sentido. Primero, revisamos cuál es la edad de la persona más vieja del mundo (118 años <a>https://www.washington.edu/news/2021/07/01/how-long-can-a-person-live-the-21st-century-may-see-a-record-breaker/</a>)

In [24]:
df_salud[df_salud['Age'] > 118]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
208,1.0,96.0,64.0,27.0,87.0,332.0,289.0,3256,0


Como solamente hay un dato, podemos quitarlo sin problema (Seguro fue un error de digitación). Tomamos los datos de edad menor a 118.

In [25]:
df_salud = df_salud[df_salud['Age'] < 118]

Veamos ahora 

In [26]:
df_salud.Outcome.value_counts().to_frame()

,Outcome
0,261
1,130


Ahora, nos damos cuenta que la variables <i>Outcome</i> no es numérica, sino booleana. Hacemos la transformación,

In [27]:
df_salud['Outcome'].astype(bool)
df_salud.dtypes

Pregnancies                 float64
Glucose                     float64
BloodPressure               float64
SkinThickness               float64
Insulin                     float64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                      object
dtype: object

In [28]:
df_salud

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
3,1.0,89.0,66.0,23.0,94.0,281.0,167.0,21,0
4,0.0,137.0,40.0,35.0,168.0,431.0,2288.0,33,1
6,3.0,78.0,50.0,32.0,88.0,31.0,248.0,26,1
8,2.0,197.0,70.0,45.0,543.0,305.0,158.0,53,1
13,1.0,189.0,60.0,23.0,846.0,301.0,398.0,59,1
...,...,...,...,...,...,...,...,...,...
753,0.0,181.0,88.0,44.0,510.0,433.0,222.0,26,1
755,1.0,128.0,88.0,39.0,110.0,365.0,1057.0,37,1
760,2.0,88.0,58.0,26.0,16.0,284.0,766.0,22,0
763,10.0,101.0,76.0,48.0,180.0,329.0,171.0,63,0


In [29]:
profiling =pandas_profiling.ProfileReport(df_salud)
profiling.to_file("salud_profile_limpieza.html") 

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]